## Pytorch のクイックスタート

各 API に関しては公式ドキュメントを参照することを推奨。


### ライブラリの import

使うやつ import するだけ。
isort などを用いて、見やすいように import する。


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [2]:
torch.cuda.is_available()


True

### データセットの作成

自分が解きたい課題がある場合(応用)はカスタムのデータセットを用意する。
提案手法を確かめたり、簡単なデータを使ったりする場合は PyTorch が提供するデータセットなどを利用する。


In [3]:
training_data = datasets.FashionMNIST(
    root="/data", train=True, download=True, transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="/data", train=False, download=True, transform=ToTensor()
)


### データローダーの作成

基本的にデータセットをバッチに分けて DataLoader というものを作成する。学習時に使用する。


In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break


Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


### モデルの作成

モデルは nn.Module を継承して作成する。構造と伝播方式を決める。


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### 損失関数と最適化手法の作成

損失関数は基本的に nn.~Loss を組み合わせて作成。カスタムの損失関数を設計している人もいる。

最適化手法に関しては Adam を使いがち、スケジューリングなどもする。


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


### 学習時の関数を作成

関数化することを推奨。dataloader、model、loss function、optimizer を使用。

（型をつけても**call**()の引数と戻り値は補完されないっぽい？）


In [7]:
def train(
    dataloader: DataLoader,
    model: NeuralNetwork,
    loss_fn: nn.CrossEntropyLoss,
    optimizer: torch.optim.SGD,
) -> None:
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred: torch.Tensor = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, cur = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{cur:>5d} / {size:>5d}]")


### 推論時の関数を作成

関数化することを推奨。dataloader、model、loss function を使用。


In [8]:
def test(
    dataloader: DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss
) -> None:
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred: torch.Tensor = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )


### 学習と推論


In [9]:
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")


Epoch 1
-------------------------------
loss: 2.293121 [   64 / 60000]
loss: 2.281806 [ 6464 / 60000]
loss: 2.258869 [12864 / 60000]
loss: 2.262832 [19264 / 60000]
loss: 2.257825 [25664 / 60000]
loss: 2.226212 [32064 / 60000]
loss: 2.235760 [38464 / 60000]
loss: 2.212218 [44864 / 60000]
loss: 2.203293 [51264 / 60000]
loss: 2.166321 [57664 / 60000]
Test Error: 
 Accuracy: 34.9%, Avg loss: 2.166673 

Epoch 2
-------------------------------
loss: 2.174085 [   64 / 60000]
loss: 2.163100 [ 6464 / 60000]
loss: 2.104940 [12864 / 60000]
loss: 2.131493 [19264 / 60000]
loss: 2.084498 [25664 / 60000]
loss: 2.028997 [32064 / 60000]
loss: 2.056357 [38464 / 60000]
loss: 1.989269 [44864 / 60000]
loss: 1.991259 [51264 / 60000]
loss: 1.917779 [57664 / 60000]
Test Error: 
 Accuracy: 58.2%, Avg loss: 1.918446 

Epoch 3
-------------------------------
loss: 1.945097 [   64 / 60000]
loss: 1.912480 [ 6464 / 60000]
loss: 1.804423 [12864 / 60000]
loss: 1.856594 [19264 / 60000]
loss: 1.733661 [25664 / 60000]
l

### 学習結果の保存

保存し忘れたり、上書きしたりすると実験がパーになるので忘れずに。


In [10]:
torch.save(model.state_dict(), "/data/FashionMNIST/model.pth")
print("Saved PyTorch Model State to model.pth")


Saved PyTorch Model State to model.pth


### 保存した重みの読み込み


In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("/data/FashionMNIST/model.pth"))


<All keys matched successfully>

### 結果の詳細を確認

実際の正答を確認し、可視化などをする。


In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred: torch.Tensor = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Ankle boot", Actual: "Ankle boot"
